In [ ]:
from datasets import load_dataset, DatasetDict

ds_names = [
    "capitals",
    "hemisphere",
    "population",
    "sciq",
    "sentiment",
    "nli",
    "authors",
    "addition",
    "subtraction",
    "multiplication",
    "modularaddition",
    "squaring",
]

In [ ]:
from collections import defaultdict
from tqdm.auto import tqdm

from elk_generalization.datasets.loader_utils import load_quirky_dataset


results = defaultdict(dict)
for ds_name in tqdm(ds_names):
    ds_id = f"EleutherAI/quirky_{ds_name}_raw"
    ds = load_quirky_dataset(ds_id).with_format("numpy")
    easy_ds = load_quirky_dataset(
                ds_id,
                character="Alice",
                max_difficulty_quantile=0.25,
                min_difficulty_quantile=0.0
            ).with_format("numpy")
    hard_ds = load_quirky_dataset(
                ds_id,
                character="Alice",
                max_difficulty_quantile=1.0,
                min_difficulty_quantile=0.75
            ).with_format("numpy")

    assert isinstance(ds, DatasetDict) and isinstance(easy_ds, DatasetDict) and isinstance(hard_ds, DatasetDict)
    for split in ("train", "test", "validation"):
        results[ds_name][f"{split}_len"] = len(ds[split])

        results[ds_name][f"{split}_frac_hard"] = 2 * len(hard_ds[split]) / len(ds[split])  # 2 because alice and bob
        results[ds_name][f"{split}_frac_easy"] = 2 * len(easy_ds[split]) / len(ds[split])
        
        # calculate alice/bob agreement rate
        # and a crosstab of alices/bobs labels
        al = ds[split]["alice_label"]
        bl = ds[split]["bob_label"]
        
        results[ds_name][f"{split}_agreement"] = (al == bl).mean()
        results[ds_name][f"{split}_alice_balance"] = (al == 1).mean()
        results[ds_name][f"{split}_bob_balance"] = (bl == 1).mean()
        results[ds_name][f"{split}_both_true"] = ((al == 1) & (bl == 1)).mean()
        results[ds_name][f"{split}_both_false"] = ((al == 0) & (bl == 0)).mean()
        results[ds_name][f"{split}_alice_true"] = ((al == 1) & (bl == 0)).mean()
        results[ds_name][f"{split}_bob_true"] = ((al == 0) & (bl == 1)).mean()

        results[ds_name][f"{split}_hard_agreement"] = (hard_ds[split]["alice_label"] == hard_ds[split]["bob_label"]).mean()
        results[ds_name][f"{split}_easy_agreement"] = (easy_ds[split]["alice_label"] == easy_ds[split]["bob_label"]).mean()
        results[ds_name][f"{split}_hard_alice_balance"] = (hard_ds[split]["alice_label"] == 1).mean()
        results[ds_name][f"{split}_hard_bob_balance"] = (hard_ds[split]["bob_label"] == 1).mean()
        results[ds_name][f"{split}_easy_alice_balance"] = (easy_ds[split]["alice_label"] == 1).mean()
        results[ds_name][f"{split}_easy_bob_balance"] = (easy_ds[split]["bob_label"] == 1).mean()

In [ ]:
import matplotlib.pyplot as plt

# show alice_balance, bob_balance, agreement, hard_agreement, easy_agreement as barplots with a shared y-axis

values = [
    "train_alice_balance",
    "train_bob_balance",
    "train_agreement",
    # "train_hard_agreement",
    # "train_easy_agreement",
]

fig, axes = plt.subplots(1, len(values), figsize=(15, 5), sharey=True)
axes[0].set_ylabel("proportion")
for ax, value in zip(axes, values):
    ax.yaxis.grid(True, alpha=0.5)
    ax.bar(
        range(len(results)),
        [results[ds_name][value] for ds_name in ds_names],
    )
    ax.set_xticks(
        range(len(results)),
    )
    ax.set_xticklabels(
        [ds_name.removesuffix("_increment0") for ds_name in ds_names],
        rotation=90,
    )
    ax.set_title(value.removeprefix("train_"))
    ax.set_ylim(0, 1)
    # turn horizontal grid lines on

plt.tight_layout()
plt.savefig("../figures/dataset_balance.pdf", bbox_inches="tight")


# value = "train_hard_bob_balance"
# plt.figure(figsize=(10, 5))
# plt.bar(
#     range(len(results)),
#     [results[ds_name][value] for ds_name in ds_names],
# )
# plt.xticks(
#     range(len(results)),
#     [ds_name for ds_name in ds_names],
#     rotation=90,
# )
# # plt.ylim(0,10000)
# plt.ylabel(value)
# plt.tight_layout()


In [ ]:
# plot train_len, validation_len, and test_len as a barplot with hues for split

import seaborn as sns
import pandas as pd

df = pd.DataFrame(
    [
        {
            "dataset": ds_name,
            "split": split,
            "value": results[ds_name][f"{split}_len"],
        }
        for ds_name in ds_names
        for split in ("train", "validation", "test")
    ]
)

plt.figure(figsize=(7, 3))

sns.barplot(
    data=df,
    x="dataset",
    y="value",
    hue="split",
    errorbar=None,
)
plt.ylabel("Number of examples")
plt.xticks(
    range(len(results)),
    [ds_name.removesuffix("_increment0") for ds_name in ds_names],
    rotation=45,
    # align the labels with the bars
    ha="right",
)
plt.semilogy()
# grid lines for y
plt.gca().yaxis.grid(True, alpha=0.5)
plt.savefig("../figures/dataset_sizes.pdf", bbox_inches="tight")

In [ ]:
# take each train crosstab and plot it as a heatmap

import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(4, 3, figsize=(8, 12), sharex=True, sharey=True)

for ax, ds_name in zip(axes.flatten(), ds_names):
    plt.sca(ax)
    plt.title(ds_name.removesuffix("_increment0"))

    
    data = np.array([
        [
            results[ds_name]["train_both_true"],
            results[ds_name]["train_bob_true"],
        ],
        [
            results[ds_name]["train_alice_true"],
            results[ds_name]["train_both_false"],
        ],
    ])

    # Add text annotations
    for i in range(2):
        for j in range(2):
            plt.text(j, i, str(round(data[i, j], 2)), ha='center', va='center', color='black', fontsize=12)

    plt.imshow(data, cmap='Blues', interpolation=None, vmin=0, vmax=1)

    if ds_name == "multiplication_increment0":
        # Add axis labels
        plt.xlabel('Alice')
        plt.xticks([0, 1], ['True', 'False'])
        plt.ylabel('Bob')
        plt.yticks([0, 1], ['True', 'False'])

# remove unused axes
for ax in axes.flatten()[len(ds_names):]:
    ax.axis("off")

plt.suptitle("Truthful and untruthful label cross-tabulations")
plt.tight_layout()
plt.savefig("../figures/truthful_and_untruthful_label_cross_tabulations.pdf")
plt.show()
